In [1]:
# ========== 1) Imports & Config ==========
import os
import math
import itertools
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unzip "/content/drive/MyDrive/CNN_potato/PotatoDiseaseClassification-main (1) (1).zip"

Archive:  /content/drive/MyDrive/CNN_potato/PotatoDiseaseClassification-main (1) (1).zip
edfb2a651fc61ba266bf20f155930cbe3a1e7c2b
replace PotatoDiseaseClassification-main/Potato Disease Classification.ipynb? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import os

# List all items in /content
print(os.listdir("/content"))

# If "PotatoDiseaseClassification-main" exists, check inside
print(os.listdir("/content/PotatoDiseaseClassification-main"))


['.config', 'drive', 'PotatoDiseaseClassification-main', 'sample_data']
['Potato Disease Classification.ipynb', 'leaves-images-dataset', 'README.md']


In [ ]:
train_dir = "/content/PotatoDiseaseClassification-main/dataset/leaves-images-dataset/train"
val_dir = "/content/PotatoDiseaseClassification-main/dataset/leaves-images-dataset/validation"
test_dir = "/content/PotatoDiseaseClassification-main/dataset/leaves-images-dataset/test"


In [ ]:
data_augmentation = keras.Sequential(
    [
        keras.Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),  # Define input shape using Input layer
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(factor=0.1),
        layers.RandomZoom(height_factor=0.1, width_factor=0.1),
    ]
)

In [ ]:
# Resize and rescale
resize_and_rescale = keras.Sequential([
    layers.Resizing(IMG_SIZE[0], IMG_SIZE[1]),
    layers.Rescaling(1./255)
])

# Apply augmentation and prepare datasets
def prepare(ds, shuffle=False, augment=False):
    # Apply resizing and rescaling
    ds = ds.map(lambda x, y: (resize_and_rescale(x), y),
                num_parallel_calls=AUTOTUNE)

    if shuffle:
        ds = ds.shuffle(1000)

    # Apply data augmentation
    if augment:
        ds = ds.map(lambda x, y: (data_augmentation(x, training=True), y),
                    num_parallel_calls=AUTOTUNE)

    # Cache and prefetch for performance
    return ds.cache().prefetch(buffer_size=AUTOTUNE)

train_ds = prepare(train_ds, shuffle=True, augment=True)
val_ds = prepare(val_ds)
test_ds = prepare(test_ds)

In [ ]:
# Create the model
num_classes = len(class_names)

model = keras.Sequential([
    resize_and_rescale,
    data_augmentation,
    layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(128, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential_5 (Sequential)       │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_4 (Sequential)       │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,305,027 (12.61 MB)

 Trainable params: 3,305,027 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Train the model
history = model.fit(
    train_ds,
    epochs=EPOCHS,
    validation_data=val_ds
)

Epoch 1/25
54/54 ━━━━━━━━━━━━━━━━━━━━ 59s 1s/step - accuracy: 0.4654 - loss: 0.8858 - val_accuracy: 0.4417 - val_loss: 0.9555
Epoch 2/25
54/54 ━━━━━━━━━━━━━━━━━━━━ 58s 1s/step - accuracy: 0.4646 - loss: 0.8857 - val_accuracy: 0.4417 - val_loss: 0.9555
Epoch 3/25
54/54 ━━━━━━━━━━━━━━━━━━━━ 59s 1s/step - accuracy: 0.4646 - loss: 0.8856 - val_accuracy: 0.4417 - val_loss: 0.9555
Epoch 4/25
54/54 ━━━━━━━━━━━━━━━━━━━━ 57s 1s/step - accuracy: 0.4646 - loss: 0.8855 - val_accuracy: 0.4417 - val_loss: 0.9555
Epoch 5/25
54/54 ━━━━━━━━━━━━━━━━━━━━ 57s 1s/step - accuracy: 0.4646 - loss: 0.8854 - val_accuracy: 0.4417 - val_loss: 0.9555
Epoch 6/25
54/54 ━━━━━━━━━━━━━━━━━━━━ 59s 1s/step - accuracy: 0.4714 - loss: 0.8853 - val_accuracy: 0.4417 - val_loss: 0.9555
Epoch 7/25
54/54 ━━━━━━━━━━━━━━━━━━━━ 58s 1s/step - accuracy: 0.4714 - loss: 0.8852 - val_accuracy: 0.4417 - val_loss: 0.9555
Epoch 8/25
54/54 ━━━━━━━━━━━━━━━━━━━━ 57s 1s/step - accuracy: 0.4682 - loss: 0.8852 - val_accuracy: 0.4417 - val_loss: